In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -q transformers torch

In [3]:
from transformers import pipeline
import pandas as pd
import numpy as np


In [4]:
# Percorsi dei file CSV
path_netflix = "/content/drive/Shareddrives/information project/ANALISI X (Twitter)/Fase 2 - Scraping twitter/tweets_netflix.csv"
path_svb = "/content/drive/Shareddrives/information project/ANALISI X (Twitter)/Fase 2 - Scraping twitter/tweets_svb.csv"

# Caricamento dataset
tweets_netflix = pd.read_csv(path_netflix)
tweets_svb = pd.read_csv(path_svb)

# Controllo rapido
print("Netflix dataset:")
display(tweets_netflix.head())

print("\nSVB dataset:")
display(tweets_svb.head())


Netflix dataset:


,id,text,twitterUrl,retweetCount,replyCount,likeCount,quoteCount,isReply,date
0,1516532661803573258,#netflix 👀👀👀 this stock needs more than a pray...,https://twitter.com/live2beingu/status/1516532...,28,11,156,2,False,2022-04-19
1,1516544297147179009,◆初の会員減\nさきほどのNETFLIX決算。3月の有料会員数は2.21億人で、昨年末比で微...,https://twitter.com/goto_finance/status/151654...,382,6,2201,62,False,2022-04-19
2,1516511437354455041,Netflix lost 200K subscribers in Q1 and expect...,https://twitter.com/screentime/status/15165114...,3436,1436,37573,7594,False,2022-04-19
3,1516553092837396490,"Netflix CEO now says he’s open to a cheaper, a...",https://twitter.com/verge/status/1516553092837...,103,87,842,169,False,2022-04-19
4,1516511634298228738,Netflix had to spend a full page of its invest...,https://twitter.com/pkafka/status/151651163429...,60,8,188,16,False,2022-04-19



SVB dataset:


,id,text,twitterUrl,retweetCount,replyCount,likeCount,quoteCount,isReply,date
0,1634329302845079555,There is more than $22 Trillion in the U.S. ba...,https://twitter.com/gaborgurbacs/status/163432...,1780,307,4956,336,False,2023-03-10
1,1634029130923794435,Silicon Valley Bank in the US is struggling fo...,https://twitter.com/AdityaD_Shah/status/163402...,478,367,2323,68,False,2023-03-10
2,1634306547638566916,"Earlier today, America's 16th largest bank, Si...",https://twitter.com/itsDanny_V/status/16343065...,487,57,1657,57,False,2023-03-10
3,1634227592227336195,🏦📉 SVB Crash Explained📉🏦\n\nSilicon Valley Ban...,https://twitter.com/FabiusMercurius/status/163...,2744,178,8627,271,False,2023-03-10
4,1634306931870375936,Silicon Valley Bank experienced $42 BILLION in...,https://twitter.com/JosephPolitano/status/1634...,264,36,1355,102,False,2023-03-10


In [5]:
# Modello multilingua con sentiment (1-5 stelle)
distilbert_multi_sentiment = pipeline(
    "sentiment-analysis",
    model="nlptown/bert-base-multilingual-uncased-sentiment"
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/669M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cpu


In [6]:
batch_size = 16
scores = []

for i in range(0, len(tweets_netflix), batch_size):
    batch_texts = tweets_netflix["text"][i:i+batch_size].tolist()
    batch_results = distilbert_multi_sentiment(batch_texts)

    for r in batch_results:
        label = r["label"]
        stars = int(label[0])
        if stars <= 2:
            scores.append(- (3 - stars)/2)
        elif stars == 3:
            scores.append(0.0)
        else:
            scores.append((stars - 3)/2)

tweets_netflix["distilbert_multi_score"] = scores


In [7]:
batch_size = 16
scores = []

for i in range(0, len(tweets_svb), batch_size):
    batch_texts = tweets_svb["text"][i:i+batch_size].tolist()
    batch_results = distilbert_multi_sentiment(batch_texts)

    for r in batch_results:
        label = r["label"]
        stars = int(label[0])
        if stars <= 2:
            scores.append(- (3 - stars)/2)
        elif stars == 3:
            scores.append(0.0)
        else:
            scores.append((stars - 3)/2)

tweets_svb["distilbert_multi_score"] = scores

In [8]:
def distilbert_multi_score(text):
    try:
        result = distilbert_multi_sentiment(text[:512])[0]  # tronca a 512 token
        label = result["label"]  # es. '1 star', '5 stars'
        stars = int(label[0])

        # conversione stelle → score continuo [-1, +1]
        if stars <= 2:
            return - (3 - stars)/2  # negativo
        elif stars == 3:
            return 0.0               # neutro
        else:
            return (stars - 3)/2     # positivo
    except:
        return 0.0


In [9]:
# Netflix
tweets_netflix["distilbert_multi_score"] = tweets_netflix["text"].apply(distilbert_multi_score)

# SVB
tweets_svb["distilbert_multi_score"] = tweets_svb["text"].apply(distilbert_multi_score)


In [10]:
tweets_netflix[["text", "distilbert_multi_score"]].head()
tweets_svb[["text", "distilbert_multi_score"]].head()


,text,distilbert_multi_score
0,There is more than $22 Trillion in the U.S. ba...,-1.0
1,Silicon Valley Bank in the US is struggling fo...,-0.5
2,"Earlier today, America's 16th largest bank, Si...",-1.0
3,🏦📉 SVB Crash Explained📉🏦\n\nSilicon Valley Ban...,-1.0
4,Silicon Valley Bank experienced $42 BILLION in...,-1.0


In [18]:
import os

base_dir = "/content/drive/Shareddrives/information project/ANALISI X (Twitter)/Fase 3 - Sentiment analysis"

# crea la cartella se non esiste
os.makedirs(base_dir, exist_ok=True)

out_netflix = f"{base_dir}/tweets_netflix_distilbert_multi.csv"
out_svb = f"{base_dir}/tweets_svb_distilbert_multi.csv"

tweets_netflix.to_csv(out_netflix, index=False)
tweets_svb.to_csv(out_svb, index=False)

print("CSV salvati correttamente ")


CSV salvati correttamente 
